CONTENTS LIST:
- Wrangling customer dataset
- Consistency checks
- Merging customer data with orders and products data

#01 Importing libraries and data

In [3]:
#importing libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [4]:
#importing data
path = '/Users/gingermoore/Documents/04-2025 Instacart Basket Analysis'
df_customer = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [5]:
df_customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
df_customer.shape

(206209, 10)

#02 Data Wrangling

In [8]:
#Renaming columns
df_customer.rename(columns = {'Surnam' : 'Surname'}, inplace = True)

In [9]:
df_customer.rename(columns = {'STATE' : 'State'}, inplace = True)

In [10]:
df_customer.rename(columns = {'n_dependants' : 'dependents'}, inplace = True)

In [11]:
df_customer.head()

,user_id,First Name,Surname,Gender,State,Age,date_joined,dependents,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


After reviewing the brief, it does not appear to be important to know the names of Instacart customers for the purpose of our analysis. I'm going to drop the first name and surname columns.

In [13]:
df_customer_2 = df_customer.drop(columns = ['First Name', 'Surname'])

In [14]:
df_customer_2.head()

,user_id,Gender,State,Age,date_joined,dependents,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


#03 Consistency and Quality Checks

In [16]:
df_customer_2.describe()

,user_id,Age,dependents,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [17]:
df_customer_2.dtypes

user_id         int64
Gender         object
State          object
Age             int64
date_joined    object
dependents      int64
fam_status     object
income          int64
dtype: object

In [18]:
#Checking for mixed type data
for col in df_customer_2.columns.tolist():
  weird = (df_customer_2[[col]].map(type) != df_customer_2[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer_2[weird]) > 0:
    print (col)

I'm going to assume this worked and there are no mixed data types. Moving on.

In [20]:
#Checking for missing values
df_customer_2.isnull().sum()

user_id        0
Gender         0
State          0
Age            0
date_joined    0
dependents     0
fam_status     0
income         0
dtype: int64

No missing values! Moving on.

In [22]:
#checking for full duplicates
df_customer_2.shape

(206209, 8)

In [23]:
#creating new dataframe for duplicate check
df_dups = df_customer_2[df_customer_2.duplicated()]

In [24]:
df_dups

,user_id,Gender,State,Age,date_joined,dependents,fam_status,income


No complete duplicates!

#04 Combining data

In [27]:
df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_w_flags.pkl'))

In [28]:
df_ords_prods.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,...,reordered,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order_total,spending_flag,median_days,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,0,Mid-range product,Regularly busy,Most orders,32,Regular customer,6.935811,High spender,8.0,Non-frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,Mid-range product,Regularly busy,Average orders,32,Regular customer,6.935811,High spender,8.0,Non-frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,0,Mid-range product,Busiest days,Average orders,5,New customer,7.930208,High spender,8.0,Non-frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,0,Mid-range product,Least busy days,Most orders,3,New customer,4.972414,High spender,9.0,Non-frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,1,Mid-range product,Least busy days,Average orders,3,New customer,4.972414,High spender,9.0,Non-frequent customer


In [29]:
df_ords_prods.shape

(32399732, 23)

In [30]:
df_ords_prods['user_id'].dtypes

dtype('int64')

In [31]:
ords_prods_cust = df_customer_2.merge(df_ords_prods, on = 'user_id', indicator = True)

In [32]:
ords_prods_cust.head()

,user_id,Gender,State,Age,date_joined,dependents,fam_status,income,product_id,product_name,...,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order_total,spending_flag,median_days,frequency_flag,_merge
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,196,Soda,...,Mid-range product,Busiest days,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer,both
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,196,Soda,...,Mid-range product,Regularly busy,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer,both
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,196,Soda,...,Mid-range product,Busiest days,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer,both
3,26711,Female,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,...,Low-range product,Regularly busy,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer,both
4,26711,Female,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,...,Low-range product,Least busy days,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer,both


In [33]:
ords_prods_cust.columns

Index(['user_id', 'Gender', 'State', 'Age', 'date_joined', 'dependents',
       'fam_status', 'income', 'product_id', 'product_name', 'aisle_id',
       'department_id', 'prices', 'order_id', 'orders_chronological',
       'orders_day_of_week', 'order_hour_of_day', 'days_since_prior_order',
       'is_new', 'add_to_cart_order', 'reordered', 'price_range',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_order_total', 'spending_flag', 'median_days', 'frequency_flag',
       '_merge'],
      dtype='object')

In [34]:
ords_prods_cust['_merge'].value_counts(dropna = False)

_merge
both          32399732
left_only            0
right_only           0
Name: count, dtype: int64

In [35]:
df_ords_prods_cust = ords_prods_cust.drop(columns = '_merge')

In [36]:
df_ords_prods_cust.head()

,user_id,Gender,State,Age,date_joined,dependents,fam_status,income,product_id,product_name,...,reordered,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order_total,spending_flag,median_days,frequency_flag
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,196,Soda,...,0,Mid-range product,Busiest days,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,196,Soda,...,1,Mid-range product,Regularly busy,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,196,Soda,...,1,Mid-range product,Busiest days,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer
3,26711,Female,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,...,0,Low-range product,Regularly busy,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer
4,26711,Female,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,...,1,Low-range product,Least busy days,Most orders,8,New customer,7.988889,High spender,19.0,Frequent customer


In [37]:
#Exporting data
df_ords_prods_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cust.pkl'))

In [38]:
df_ords_prods_cust.describe()

,user_id,Age,dependents,income,product_id,aisle_id,department_id,prices,order_id,orders_chronological,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,max_order,avg_order_total,median_days
count,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.032399e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07,3.239973e+07
mean,1.029367e+05,4.946528e+01,1.501891e+00,9.941685e+04,2.559904e+04,7.119081e+01,9.918830e+00,7.790994e+00,1.710746e+06,1.714260e+01,2.738865e+00,1.342514e+01,1.110395e+01,8.352604e+00,5.895835e-01,3.305268e+01,7.790994e+00,1.039753e+01
std,5.946623e+04,1.848558e+01,1.118867e+00,4.300669e+04,1.408498e+04,3.821194e+01,6.281516e+00,4.241809e+00,9.872994e+05,1.753544e+01,2.090089e+00,4.246397e+00,8.779012e+00,7.127110e+00,4.919093e-01,2.515525e+01,7.348239e-01,7.131719e+00
min,1.000000e+00,1.800000e+01,0.000000e+00,2.590300e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,5.142000e+04,3.300000e+01,1.000000e+00,6.699600e+04,1.354100e+04,3.100000e+01,4.000000e+00,4.200000e+00,8.559450e+05,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,3.000000e+00,0.000000e+00,1.300000e+01,7.378488e+00,6.000000e+00
50%,1.026140e+05,4.900000e+01,2.000000e+00,9.660800e+04,2.530500e+04,8.300000e+01,9.000000e+00,7.400000e+00,1.711052e+06,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,6.000000e+00,1.000000e+00,2.600000e+01,7.811946e+00,8.000000e+00
75%,1.543880e+05,6.500000e+01,3.000000e+00,1.278840e+05,3.794700e+04,1.070000e+02,1.600000e+01,1.130000e+01,2.565499e+06,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,1.100000e+01,1.000000e+00,4.700000e+01,8.229341e+00,1.300000e+01
max,2.062090e+05,8.100000e+01,3.000000e+00,5.939010e+05,4.968800e+04,1.340000e+02,2.100000e+01,2.500000e+01,3.421083e+06,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,1.450000e+02,1.000000e+00,9.900000e+01,2.320000e+01,3.000000e+01
